<a href="https://colab.research.google.com/github/kiruheh/beer_problem/blob/main/beer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 38.8 MB/s eta 0:00:00


In [3]:
# Import PuLP modeler functions
from pulp import *
import pandas
import csv

# Creates a list of all the supply nodes
Warehouses = ["A", "B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000, "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700,
}

# Creates a list of costs of each transportation path
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
]




In [5]:
with open('warehouses_supply.csv','w') as f:
  writer = csv.writer(f)
  writer.writerow(Warehouses)
  writer.writerow(supply.values())

with open('bars_demand.csv','w') as f:
  writer = csv.writer(f)
  writer.writerow(Bars)
  writer.writerow(demand.values())

with open('transport_cost.csv','w') as f:
  writer = csv.writer(f)
  writer.writerow(['Warehouse'] + Bars)
  for i, cost in enumerate(costs):
    writer.writerow([Warehouses[i]] + cost)

In [63]:
import pandas as pd

costs = pd.read_csv('transport_cost.csv')
supplys = pd.read_csv('warehouses_supply.csv')
demands = pd.read_csv('bars_demand.csv')

In [52]:
WH_names = costs['Warehouse'].values.tolist()
B_names = costs.columns.values.tolist()[1:]

In [64]:
main = 0
model = LpProblem(name='Beer problem', sense=LpMinimize)
b = True
Warehouses = []
X = {}
for bar in B_names:
  demand = 0

  for i, warehouse in enumerate(WH_names):
    x = LpVariable(name=f"x{bar}_{warehouse}", lowBound=0, cat = 'Integer')
    demand += x
    main += x * int(costs[bar][i])
    X[(bar, warehouse)] = x

  model += (demand >= int(demands[bar][0]), f"{bar} bar demand")

for warehouse in WH_names:
  supply = 0
  for bar in B_names:
    supply += X[((bar, warehouse))]

  model += (supply <= int(supplys[warehouse][0]), f"{warehouse} warehouse supply")

model += main
model

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Beer_problem:
MINIMIZE
2*x1_A + 3*x1_B + 4*x2_A + 1*x2_B + 5*x3_A + 3*x3_B + 2*x4_A + 2*x4_B + 1*x5_A + 3*x5_B + 0
SUBJECT TO
1_bar_demand: x1_A + x1_B >= 500

2_bar_demand: x2_A + x2_B >= 900

3_bar_demand: x3_A + x3_B >= 1800

4_bar_demand: x4_A + x4_B >= 200

5_bar_demand: x5_A + x5_B >= 700

A_warehouse_supply: x1_A + x2_A + x3_A + x4_A + x5_A <= 1000

B_warehouse_supply: x1_B + x2_B + x3_B + x4_B + x5_B <= 4000

VARIABLES
0 <= x1_A Integer
0 <= x1_B Integer
0 <= x2_A Integer
0 <= x2_B Integer
0 <= x3_A Integer
0 <= x3_B Integer
0 <= x4_A Integer
0 <= x4_B Integer
0 <= x5_A Integer
0 <= x5_B Integer

In [66]:
model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in X.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 8600.0
x1_A: 300.0
x1_B: 200.0
x2_A: 0.0
x2_B: 900.0
x3_A: 0.0
x3_B: 1800.0
x4_A: 0.0
x4_B: 200.0
x5_A: 700.0
x5_B: 0.0
1_bar_demand: 0.0
2_bar_demand: 0.0
3_bar_demand: 0.0
4_bar_demand: 0.0
5_bar_demand: 0.0
A_warehouse_supply: 0.0
B_warehouse_supply: -900.0
